# Mapping MSRC CVEs to Bins

Is it possible to map [MSRC CVEs](https://msrc.microsoft.com/update-guide/vulnerability) to Windows binaries?


```{mermaid}

graph TD
    A[winbindex fa:fa-database] --> B[KBs]
    C[ms support feeds fa:fa-database] -->B
    B --> D[msrc_cve_to_bins.py]
    A --> E
    C --> E
    E[File Version Info fa:fa-file] --> D
    F[Known MSRC Tags + Titles fa:fa-tag] --> D
    D --> G(cve to bin fa:fa-map)

```

## Setup

### import cvedata

In [ ]:
from cvedata.winbindex import get_winbindex_desc_to_bin_map, WINBINDEX_GITHUB_URL, get_winbindex_kbs_to_bin_map
from cvedata.win_verinfo import get_verinfo_desc_to_bins_json
from cvedata.msrc_pandas import get_msrc_tags, get_msrc_titles, get_msrc_cvrf_pandas_df, MSRC_CVRF_PANDAS_FULL,MSRC_CVRF_PANDAS
from cvedata.msrc_known_bins import KNOWN_TAG_TO_BIN_MAP, KNOWN_TITLE_TO_BIN_MAP
from cvedata.ms_feed_kbs import get_ms_kb_to_bins_json


from cvedata.msrc_pandas import clean_impact
from cvedata.msrc_cve_to_bins import clean_tag,get_msrc_all_desc_to_bins

In [ ]:
import pandas as pd
import difflib
from datetime import datetime

pd.set_option('display.max_rows', 20)

### Setup data sources

In [ ]:
KNOWN_TAG_TO_BIN_MAP, KNOWN_TITLE_TO_BIN_MAP # hardcoded tag/title to bin
tags_json = get_msrc_tags()
titles_json = get_msrc_titles()
verinfo_desc_to_bins = get_verinfo_desc_to_bins_json()
wv_ver = get_winbindex_desc_to_bin_map()
all_desc_to_bins = get_msrc_all_desc_to_bins() #combined wv_ver and verinfo

## Correlate Data Sources

### Build list of all known Windows binary names

In [ ]:
bin_names = []

for key in wv_ver:
    for bin in wv_ver[key]:
        bin_names.append(bin)

for key in verinfo_desc_to_bins:
    for bin in verinfo_desc_to_bins[key]:
        bin_names.append(bin)

bin_names = list(set(bin_names))

all_bin_names = {}

for bin in bin_names:
    all_bin_names[bin] = [bin]

len(all_bin_names)

### Correlate MSRC tags to Windows Binary Names

In [ ]:
def get_match_at_cutoff(key,possibilities: dict,cutoff: float = 0.6):

    if key is None:
        return []

    key = clean_tag(key)
    matches = difflib.get_close_matches(key,possibilities.keys(),n=10000,cutoff=cutoff)

    # map keys to bins
    bins = [bin for desc in matches for bin in possibilities[desc]]
    return bins
    
all_bins_df = pd.DataFrame(get_msrc_tags(),columns=['Tag'])
all_bins_df['all_bin_names'] = all_bins_df['Tag'].apply(get_match_at_cutoff, args=(all_bin_names,.75))
all_bins_df[all_bins_df['all_bin_names'].apply(lambda x: len(x) > 0)]

In [ ]:
# Save results in a dictionary for later
tags_all_bins = all_bins_df[['Tag','all_bin_names']]
tags_all_bins.set_index('Tag', inplace=True)
all_bins_d = tags_all_bins['all_bin_names'].to_dict()

In [ ]:
all_cvrf_df = pd.DataFrame.from_dict(get_msrc_cvrf_pandas_df())
all_cvrf_df = all_cvrf_df.drop('Acks',axis=1)
all_cvrf_df.head()

 ### Correlate MSRC tags to Windows File Descriptions across Similarities [.25, .45, .5, .55, .75, .9, 1]

In [ ]:
def get_tag_similarity_df(row : str,key : str,desc_to_bins: dict,col_pre : str,min_sims: list):
    """
    Builds similarity columns into Dataframe at min_sims intervals
    """

    bins = {}

    # init bins
    for min_sim in min_sims:
            bins.setdefault(min_sim,[])

    ctag1 = clean_tag(row[key]).split()

    for desc in desc_to_bins:

        # check for bad/common tag
        if len(ctag1) == 0:
            break
        
        ctag2 = clean_tag(desc).split()

        if len(ctag2) == 0:
            continue
        
        s = difflib.SequenceMatcher(None,ctag1,ctag2)
        
        if s.real_quick_ratio() > min_sims[0] and s.quick_ratio() > min_sims[0]:
            
            sim = s.ratio()

            # add bins to 
            for min_sim in min_sims:                      
                if sim >= min_sim:
                    [bins[min_sim].append(bin) for bin in desc_to_bins[desc]]

    for sim_score in bins:
        row[f"{col_pre}-{sim_score}"] = bins[sim_score]

    return row

In [ ]:
key = 'Tag'
tags_sim_df = pd.DataFrame(all_cvrf_df['Tag'].unique(),columns= ['Tag'])
tags_sim_df.head()

In [ ]:
key = 'Tag'
tags_sim_verinfo_df = tags_sim_df.apply(get_tag_similarity_df,args=(key,all_desc_to_bins,'vi',[.25, .45, .5, .55, .75, .9, 1]),axis=1)
tags_sim_map = tags_sim_verinfo_df.set_index('Tag')
tags_sim_verinfo_df.head()

In [ ]:
titles_sim_df = pd.DataFrame(all_cvrf_df['Title'].unique(),columns= ['Title'])
# drop chromium titles
titles_sim_df = titles_sim_df[~titles_sim_df.Title.str.contains('Chromium CVE')]
titles_sim_df.head()

In [ ]:
key = 'Title'
titles_sim_verinfo_df = titles_sim_df.apply(get_tag_similarity_df,args=(key,all_desc_to_bins,'vi',[.25, .45, .5, .55, .75, .9, 1]),axis=1)
all_titles_sim_map = titles_sim_verinfo_df.set_index('Title')
titles_sim_verinfo_df.head(25)

## What is the number of bins on average for each similarity?

In [ ]:
tags_sim_verinfo_df.apply(lambda s: s.map(lambda x: len(x) if x else 0)).mean()

In [ ]:
titles_sim_verinfo_df.apply(lambda s: s.map(lambda x: len(x))).mean() 

### Which tag descriptions are a bit too noisy?

In [ ]:
bin_count_tags_sim_verinfo_df = tags_sim_verinfo_df.set_index('Tag')
bin_count_tags_sim_verinfo_df.apply(lambda s: s.map(lambda x: len(x) if x else 0))['vi-0.45'].sort_values(ascending=False)

### Which titles are a bit too noisy?

In [ ]:
bin_count_titles_sim_verinfo_df = titles_sim_verinfo_df.set_index('Title')
bin_count_titles_sim_verinfo_df.apply(lambda s: s.map(lambda x: len(x) if x else 0))['vi-0.45'].sort_values(ascending=False)

In [ ]:
tags_sim_map = tags_sim_verinfo_df.copy()
tags_sim_map.set_index('Tag',inplace=True)
tag_counts_df = tags_sim_map.apply(lambda s: s.map(lambda x: len(x) if x else 0))

counts = []
for col in tag_counts_df.columns:
    print(col)
    print(tag_counts_df[tag_counts_df[col] > 0].count()[col])
    counts.append(pd.Series( {col: tag_counts_df[tag_counts_df[col] > 0].count()[col]}))
    


In [ ]:
pd.concat(counts).plot.bar(figsize=(20,5), title="# Tags correlated across similarities")

In [ ]:
titles_sim_map = titles_sim_verinfo_df.copy()
titles_sim_map.set_index('Title',inplace=True)
title_counts_df = titles_sim_map.apply(lambda s: s.map(lambda x: len(x) if x else 0))

counts = []
for col in title_counts_df.columns:
    print(col)
    print(title_counts_df[title_counts_df[col] > 0].count()[col])
    counts.append(pd.Series( {col: title_counts_df[title_counts_df[col] > 0].count()[col]}))

In [ ]:
pd.concat(counts).plot.bar(figsize=(20,5), title="# of Titles correlated across similarities")

## Setup KB update data

In [ ]:
wb_feed = get_winbindex_kbs_to_bin_map()
wb_feed_d = {}

for key in wb_feed:
    wb_feed_d[key] = wb_feed[key]['updated']

In [ ]:
kb_feed_df = None
kb_feed = get_ms_kb_to_bins_json()['updated']

## Mapping CVEs to bins

In [ ]:
def cve_to_bin(row):
    cve = row.name
    tag = row['Tag']
    title = row['Title']

    bins = set()
    
    if tag and tag.lower() in KNOWN_TAG_TO_BIN_MAP:
        for bin in KNOWN_TAG_TO_BIN_MAP[tag.lower()]:
            bins.add(bin)

    if title and title.lower() in KNOWN_TITLE_TO_BIN_MAP:
        for bin in KNOWN_TITLE_TO_BIN_MAP[title.lower()]:
            bins.add(bin)

    if all_bins_d.get(tag):
        for bin in all_bins_d.get(tag):
            bins.add(bin)

    if tag and "microsoft" in tag.lower():
        tag_min = 'vi-0.55'
    else:
        tag_min = 'vi-0.45'

    if title and "microsoft" in title.lower():
        title_min = 'vi-0.55'
    else:
        title_min = 'vi-0.45'

    for bin in tags_sim_map[tag_min].loc[tag]:
        bins.add(bin)

    if titles_sim_map[title_min].get(title):
        for bin in titles_sim_map[title_min].get(title):
            bins.add(bin)

    # updated_bins    
    updated_bins = []
    for kb in row['KBs'].split():        
        kb_updated_files = kb_feed.get(kb)
        if kb_updated_files:
            updated_bins.extend(kb_updated_files)
        wb_updated_files = wb_feed_d.get(kb)
        if wb_updated_files:
            updated_bins.extend(wb_updated_files)
            wb_feed_d


    row['Bins Updated'] = list(set(updated_bins).intersection(bins))
    row['Bins'] = list(bins)

    return row

print(all_cvrf_df.columns)

bins_all_cvrf_df = all_cvrf_df.apply(cve_to_bin,axis=1)
bins_all_cvrf_df.head()

In [ ]:
bins_all_cvrf_df.sort_values(by=['Bins'], ascending=False)

### How man CVEs have bins assigned?

In [ ]:
bins_all_cvrf_df['Bins Count'] = bins_all_cvrf_df['Bins'].apply(lambda x: len(x) if x else 0)
bins_all_cvrf_df['Bins Updated Count'] = bins_all_cvrf_df['Bins Updated'].apply(lambda x: len(x) if x else 0)
bins_all_cvrf_df.head()


#### CVEs with Bins Assigned

In [ ]:
bins_all_cvrf_df[bins_all_cvrf_df['Bins Count'] > 0]

#### CVEs with Bins Assigned and Verified with KB Update

In [ ]:
bins_all_cvrf_df[bins_all_cvrf_df['Bins Updated Count'] > 0].sort_values(by='Bins Updated Count', ascending=False)

### How many verified from this month's MSRC update?

In [ ]:
year = datetime.now().year
month = datetime.now().month
bins_all_cvrf_df[bins_all_cvrf_df['Initial Release'].str.contains(f'{year}-{month}') & bins_all_cvrf_df['Bins Updated Count'] > 0]

## How many CVEs have KB data?

In [ ]:
def has_kb_data(kbs):

    has_kb_data = False

    for kb in kbs.split():
        if kb_feed.get(kb) and (len(kb_feed.get(kb)['updated']) > 0):
            has_kb_data = True
            break
        if wb_feed_d.get(kb) and (len(wb_feed_d.get(kb)['updated']) > 0):
            has_kb_data = True
            break

bins_all_cvrf_df['has_kb'] = bins_all_cvrf_df['KBs'].apply(lambda x: len(x.split()) > 0)
bins_all_cvrf_df['has_kb_data'] = bins_all_cvrf_df['KBs'].apply(lambda x: len(x.split()) > 0)
bins_all_cvrf_df.head()

### How many CVEs have KB information but no verified bins? (cvedata is missing KB source coverage)

In [ ]:
bins_all_cvrf_df[(bins_all_cvrf_df['Initial Release'].str.contains(f"{year}-{month}")) & (bins_all_cvrf_df['Bins Updated Count'] == 0) & (bins_all_cvrf_df['has_kb'] == True)]

## For all updates, what % have assigned bins?

In [ ]:

updates = bins_all_cvrf_df['Initial Release'].unique()

results = {}

for update in updates:
    results.setdefault(update,{})
    update_df = bins_all_cvrf_df[bins_all_cvrf_df['Initial Release'].str.contains(update)]
    cve_per_update = update_df.shape[0]
    cves_with_updated_bins = update_df[update_df['Bins Updated Count'] > 0]
    cves_without_updated_bins = update_df[update_df['Bins Updated Count'] == 0]
    cves_without_updated_bins_and_has_kbs = update_df[(update_df['Bins Updated Count'] == 0) & (update_df['has_kb'] == True)]
    cves_with_bins = update_df[update_df['Bins Count'] > 0]    
    cves_without_bins = update_df[(update_df['Bins Count'] == 0)]
    
    cve_with_bins_percent = (cves_with_bins.shape[0] * 100 / cve_per_update) 
    cve_without_bins_percent = (cves_without_bins.shape[0] * 100 / cve_per_update)
    
    results[update]['with_bins'] = cves_with_bins.shape[0]
    results[update]['without_bins'] = cves_without_bins.shape[0]
    results[update]['with_updated_bins'] = cves_with_updated_bins.shape[0]
    results[update]['without_updated_bins'] = cves_without_updated_bins.shape[0]
    results[update]['without_updated_bins_has_kbs'] = cves_without_updated_bins_and_has_kbs.shape[0]
    
    
update_results_df = pd.DataFrame.from_dict(results, orient='index')
update_results_df.index = pd.to_datetime(update_results_df.index)
update_results_df = update_results_df.sort_index()
update_results_df.head()


### CVEs with Bins Assigned This Year

In [ ]:
title='CVEs with Bins Assigned This Year'
update_results_df.loc[update_results_df.index.year == year][['with_bins','without_bins']].plot.bar(figsize=(20,3), stacked=True, title=title)

### CVEs with Bins Assigned All Time

In [ ]:
title='CVEs with Bins Assigned All Time'
update_results_df[['with_bins','without_bins']].plot.bar(figsize=(20,5), stacked=True, title=title)

### CVEs with KB Verified Update Bins This Year

In [ ]:
title='CVEs with KB Verified Update Bins This Year'
update_results_df.loc[update_results_df.index.year == year][['with_updated_bins','without_updated_bins']].plot.bar(figsize=(20,3), stacked=True, title=title)

### CVEs with KB Verified Update Bins All Time

In [ ]:
title='CVEs with KB Verified Update Bins All Time'
update_results_df[['with_updated_bins','without_updated_bins']].plot.bar(figsize=(20,5), stacked=True, title=title)

### CVEs with KB Verified Update Bins That Have KB information This Year

In [ ]:
title='CVEs with KB Verified Update Bins That Have KB information This Year'
update_results_df.loc[update_results_df.index.year == year][['with_updated_bins','without_updated_bins_has_kbs']].plot.bar(figsize=(20,3), stacked=True, title=title)

### CVEs with KB Verified Update Bins That Have KB Source Info All Time

In [ ]:
title = 'CVEs with KB Verified Update Bins That Have KB Source Info All Time'
update_results_df[['with_updated_bins','without_updated_bins_has_kbs']].plot.bar(figsize=(20,5), stacked=True, title=title)

### CVEs with KB Verified Update Bins That Have KB Source Info All Time Ratio

In [ ]:
#update_results_df[['with_updated_bins','without_updated_bins_has_kbs']].apply(lambda x: [x[0] * 100 / sum(x),x[1] * 100 / sum(x) ], axis=1)
title = 'CVEs with KB Verified Update Bins That Have KB Source Info All Time Ratio'
updated_with_without_kbs_df = update_results_df[['with_updated_bins','without_updated_bins_has_kbs']]
updated_with_without_kbs_df = updated_with_without_kbs_df.div(updated_with_without_kbs_df.sum(axis=1), axis=0)
updated_with_without_kbs_df.plot.bar(figsize=(20,5), stacked=True, title=title)


## CVE mapping improvement

### What is the next best title to hardcode?

In [ ]:
bins_all_cvrf_df[bins_all_cvrf_df['Bins Count'] == 0]['Title'].value_counts()

### what is the next best title to hardcode that has KBs?

In [ ]:
bins_all_cvrf_df[(bins_all_cvrf_df['Bins Updated Count'] == 0) & (bins_all_cvrf_df['has_kb'] == True)]['Title'].value_counts()

### what is the next best tag to hardcode?

In [ ]:
bins_all_cvrf_df[bins_all_cvrf_df['Bins Count'] == 0]['Tag'].value_counts()

### what is the next best title to hardcode that has KBs?

In [ ]:
bins_all_cvrf_df[(bins_all_cvrf_df['Bins Count'] == 0) & (bins_all_cvrf_df['has_kb'] == True)]['Tag'].value_counts()

### What is the next best tag/title combo?

In [ ]:
tags_df = bins_all_cvrf_df[bins_all_cvrf_df['Bins Count'] == 0]
tags_df = tags_df.loc[tags_df['Initial Release'].str.contains('2022')]
tags_title_df = tags_df.reset_index().groupby(['Tag','Title']).aggregate(lambda x: list(set(x)))
tags_title_df = tags_title_df.rename(columns={'index': 'CVEs'})
tags_title_df['CVE Count'] = tags_title_df['CVEs'].apply(lambda x: len(x))
tags_title_df.sort_values(by=['CVE Count'], ascending=False)

### What is the next best hardcode tag title with kbs?

In [ ]:
tags_title_df[tags_title_df['has_kb'].apply(lambda x: x[0]) == True].sort_values(by=['CVE Count'], ascending=False)